In [23]:
#--**coding:utf-8**--
import pandas as pd
import numpy as np
import copy
from sklearn.linear_model import Lasso,LinearRegression,LassoCV
from sklearn.linear_model.stochastic_gradient import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer,scale,MaxAbsScaler

min_time=12

data = pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',parse_dates = True)
data_train = copy.copy(data[data['avg_online_minutes'] >= min_time]) #分析了训练数据情况，处理平均在线时长小于5min的，可预测45天的就为7天内的付费值
print(data_train.shape)

data_train = data_train.drop(['user_id','register_time'],axis=1)
print(data_train.shape)

x_train = data_train.loc[:, data_train.columns != 'prediction_pay_price']
y_train = data_train.loc[:, data_train.columns == 'prediction_pay_price']

model = Lasso() #分析了训练数据，存在大量共线，可使用L1正则化消除共线
model.fit(x_train, y_train)
print(model.coef_)
print (len(model.coef_))

none_mean = []
for i in range(len(model.coef_)):
    if abs(model.coef_[i]) < 1e-06:
        none_mean.append(x_train.columns[i])
        
print(none_mean)
print (len(none_mean))

x_train_final = x_train.drop(none_mean, axis=1)

X_train, X_test, Y_train, Y_test = train_test_split(x_train_final,y_train,test_size = 0.2, random_state = 0)
X_test_7pays = X_test['pay_price'].tolist()

scaler = StandardScaler(with_mean=False).fit(X_train.values)
#scaler = MaxAbsScaler().fit(X_train.values)
X_train = scaler.transform(X_train.values)
X_test = scaler.transform(X_test.values)

SGDModel = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01)#RandomForestRegressor(n_estimators=500)#LinearRegression()
##SGDRegressor()
#可使用网格法选参数，然而分出的验证集并不好，可使用k折交叉验证试试
SGDModel.fit(X_train, Y_train.values.ravel())
joblib.dump(SGDModel,'F:\\study\\ml\\DataSet\\Tap4fun\\Model\\train_model.m')

y_pred = SGDModel.predict(X_test)
np.sqrt(mean_squared_error(Y_test,y_pred))

(275717, 109)
(275717, 107)


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[ 6.20417558e-07  5.26177183e-06 -9.01406414e-07  2.58730669e-06
  1.26297380e-05 -1.98281769e-05 -6.20828489e-07  3.81479221e-07
  8.88581620e-07 -1.90037145e-06 -4.27988000e-03  6.40414395e-03
  1.55859797e-02 -1.18105184e-02 -7.69854644e-03  8.60990278e-03
 -5.50771528e-03  3.79048587e-03 -2.52490882e-03 -2.31504390e-03
  5.66069890e-04 -2.01501130e-03 -2.87705839e-03  4.14728745e-04
  4.31926627e-04 -9.08751381e-03 -3.67890418e-03  2.78464638e-03
  9.38689525e-03 -1.09772506e-02  1.09366161e-02 -8.66451167e-02
  0.00000000e+00 -0.00000000e+00 -1.71058284e-02 -1.73949967e-01
 -1.13415193e-01  3.97065469e-01 -3.01812253e-01 -1.97414053e+00
  0.00000000e+00  6.63390184e-01 -2.30785762e-01  2.48238629e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -1.15786644e-01
  0.00000000e+00 -0.00000000e+00 -2.47359289e+00 -2.98786086e+00
 -6.80022616e+00  9.47380190e-01  0.00000000e+00  1.96626202e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000

200.56181869614113

In [ ]:
X_test.shape

In [ ]:
# data[data['avg_online_minutes'] >12].shape

# data['avg_online_minutes'] .describe()

# data['avg_online_minutes'] .describe(percentiles=[0.25,0.5,0.75,0.8,0.9,0.95,0.97,0.975])

# data[data['prediction_pay_price'] > 0].shape[0]/data.shape[0]

# data[data['prediction_pay_price'] > 0]['avg_online_minutes'].describe()

# data[data['pay_price'] > 0]['avg_online_minutes'].describe()

# data[data['avg_online_minutes'] >10].shape

# data[(data['avg_online_minutes'] >7)  & (data['pay_price'] > 0)].shape

# from sklearn.metrics import mean_squared_error

# y_pred = SGDModel.predict(X_test)
# np.sqrt(mean_squared_error(Y_test,y_pred))

# X_test.shape

In [33]:
# y_pred = SGDModel.predict(X_test)
print (Y_train.values)
for i in range(len(y_pred)):
    if y_pred[i] < X_test_7pays[i]: #判断45天的预测值若比前7天的还小，则使用前7天的付费金额作为预测值
        y_pred[i] = X_test_7pays[i]

for i in range(len(y_pred)):
    if y_pred[i] < 0:
        y_pred[i] = 0
#     elif y_pred[i] < 1.4:
#         y_pred[i] = 0.99
print("Root Mean squared error: %.2f"
      % mean_squared_error(Y_test, y_pred) ** 0.5)
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(Y_test, y_pred))

data1 = pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_test.csv',parse_dates=True)
print (data1.shape)
data_test1 = copy.copy(data1[data1['avg_online_minutes'] < min_time])
data_test2 = copy.copy(data1[data1['avg_online_minutes'] >= min_time])
data_test2_7pays = data_test2['pay_price'].tolist()

#看了训练数据情况，处理平均在线时长小于5min的，可预测45天的就为7天内的付费值
data_test_part1 = data_test1[['user_id','pay_price']]
data_test_part1.rename(columns={'pay_price':'prediction_pay_price'},inplace = True)
data_test_part1.to_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part1.csv')

data_test2_id = data_test2['user_id'].values
data_test2_id_final = pd.DataFrame(data_test2_id,columns={'user_id'})
data_test2 = data_test2.drop(['user_id','register_time'],axis=1)

data_test2_final = data_test2.drop(none_mean, axis=1)
data_test2_final = scaler.transform(data_test2_final.values)
data_test2_pred = SGDModel.predict(data_test2_final)

for i in range(len(data_test2_pred)):
    if data_test2_pred[i] < data_test2_7pays[i]:
        data_test2_pred[i] = data_test2_7pays[i]

for i in range(len(data_test2_pred)):
    if data_test2_pred[i] < 0:
        data_test2_pred[i] = 0
#     elif data_test2_pred[i] < 1.4:
#         data_test2_pred[i] = 0.99

data_test2_pred_howmuch = pd.DataFrame(data_test2_pred,columns={'prediction_pay_price'})

data_test2_result = pd.concat([data_test2_id_final,data_test2_pred_howmuch],axis=1)
data_test2_result.to_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part2.csv')

pred_part1 = pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part1.csv',index_col=0,parse_dates=True)
pred_part2 = pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part2.csv',index_col=0,parse_dates=True)

pred = pred_part1.append(pred_part2)
print(pred.shape)
pred.to_csv('F:\\study\\ml\\DataSet\\Tap4fun\\SubMission\\pred_result99.csv',index=False)

[[  2.98]
 [  0.  ]
 [160.8 ]
 ...
 [  1.98]
 [  0.  ]
 [  0.  ]]
Root Mean squared error: 200.56
Variance score: 0.37
(828934, 108)


D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


(828934, 2)


In [25]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression,LassoCV,RidgeCV,ElasticNetCV
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score ,KFold

In [26]:
kfolds=KFold(n_splits=10,shuffle=True,random_state=13)

def cv_rmse(model):
    rmse=np.sqrt(-cross_val_score(model,X_train,Y_train,scoring='neg_mean_squared_error',cv=kfolds,n_jobs=4))
    return rmse
    

r_alphas=[0.01,0.1,0.5,1,5,10,50,100,500,1000]
def ridge_k(k):
    rmse_k=make_pipeline(RobustScaler(),
                         RidgeCV(alphas=[k],cv=kfolds)).fit(X_train,Y_train)
    return cv_rmse(rmse_k).mean()

r_scores=[]
for i in r_alphas:
    score=ridge_k(i)
    r_scores.append(score)




In [27]:
r_score=pd.DataFrame(list(zip(r_alphas,r_scores)),columns=['alpha','rmse'])

In [16]:
# plt.plot(r_score.alpha,r_score.rmse)
# plt.show()

In [28]:
r_model=make_pipeline(RobustScaler(),
                     RidgeCV(alphas=[1000],cv=kfolds)).fit(X_train,Y_train)
cv_rmse(r_model).mean()

175.3810033715518

In [29]:
r_pred=r_model.predict(X_test)
np.sqrt(mean_squared_error(Y_test,r_pred))

176.11275801254973

In [30]:
l_alpha = [ 0.0001,0.0002,0.0004,0.0007, 0.0008,0.001]
l_model=make_pipeline(RobustScaler(),
                     LassoCV(alphas=l_alpha,max_iter=5000,cv=kfolds)).fit(X_train,Y_train)


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [31]:
cv_rmse(l_model).mean()

175.5251501815137

In [21]:
# e_alphas = [0.00001,0.00005,0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
# e_l1ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.85, 0.9, 0.95, 0.99, 1]

# elastic_model=make_pipeline(RobustScaler(),
#                         ElasticNetCV(max_iter=1e5,alphas=e_alphas,l1_ratio=e_l1ratio,cv=kfolds)).fit(X_train,Y_train)

# cv_rmse(elastic_model).mean()

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


178.59660971556212

In [32]:
data_test2_pred = l_model.predict(data_test2_final)

for i in range(len(data_test2_pred)):
    if data_test2_pred[i] < data_test2_7pays[i]:
        data_test2_pred[i] = data_test2_7pays[i]

for i in range(len(data_test2_pred)):
    if data_test2_pred[i] < 0:
        data_test2_pred[i] = 0
#     elif data_test2_pred[i] < 1.4:
#         data_test2_pred[i] = 0.99

data_test2_pred_howmuch = pd.DataFrame(data_test2_pred,columns={'prediction_pay_price'})

data_test2_result = pd.concat([data_test2_id_final,data_test2_pred_howmuch],axis=1)
data_test2_result.to_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part2.csv')

pred_part1 = pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part1.csv',index_col=0,parse_dates=True)
pred_part2 = pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_result_part2.csv',index_col=0,parse_dates=True)

pred = pred_part1.append(pred_part2)
print(pred.shape)
pred.to_csv('F:\\study\\ml\\DataSet\\Tap4fun\\SubMission\\pred_result9.csv',index=False)

(828934, 2)
